In [1]:
#!pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from datasets import load_dataset

In [7]:
trivia_qa = load_dataset('trivia_qa', 'rc')

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [8]:
#print(trivia_qa['train'][:10])

In [15]:
#print(len(trivia_qa["train"]))

138384


In [13]:
#print(len(trivia_qa["validation"]))

17944


In [23]:
from datasets import Dataset, DatasetDict

In [19]:
id = []
title = []
context = []
question = []
answers = []

#for i in range(len(trivia_qa["train"])):     
for i in range(138):    
    i += 1
    for k, v in trivia_qa["train"][i].items():
        if k == 'entity_pages':
            ep = v
            for k, v in ep.items():
                if k == 'title':
                    if len(v) > 0:
                        title.append(v[-1])  # Add title
                    else:
                        title.append('dummy_title')  # Add title
                if k == 'wiki_context':
                    if len(v) > 0:
                        v = v[0]
                        context.append(v)  # Add context
                    else:
                        context.append('dummy_text')  # Add title
        if k == 'answer':
            a = v
            for k, v in v.items():
                if k == 'value':
#                    print(type(v))
                    answers.append(v)  # Add answer
        if k == 'question':
            q = v
            question.append(q)  # Add question
        if k == 'question_id':
            qid = v
            id.append(qid)  # Add id

In [25]:
# Create the Dataset object
training_dataset = Dataset.from_dict({
    "id": id,
    "title": title,
    "context": context,
    "question": question,
    "answers": answers
})

In [27]:
training_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 138
})

In [33]:
id = []
title = []
context = []
question = []
answers = []

#for i in range(len(trivia_qa["validation"])):     
for i in range(18):    
    i += 1
    for k, v in trivia_qa["validation"][i].items():
        if k == 'entity_pages':
            ep = v
            for k, v in ep.items():
                if k == 'title':
                    if len(v) > 0:
                        title.append(v[-1])  # Add title
                    else:
                        title.append('dummy_title')  # Add title
                if k == 'wiki_context':
                    if len(v) > 0:
                        v = v[0]
                        context.append(v)  # Add context
                    else:
                        context.append('dummy_text')  # Add title
        if k == 'answer':
            a = v
            for k, v in v.items():
                if k == 'value':
#                    print(type(v))
                    answers.append(v)  # Add answer
        if k == 'question':
            q = v
            question.append(q)  # Add question
        if k == 'question_id':
            qid = v
            id.append(qid)  # Add id

In [35]:
# Create the Dataset object
validation_dataset = Dataset.from_dict({
    "id": id,
    "title": title,
    "context": context,
    "question": question,
    "answers": answers
})

In [37]:
validation_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 18
})

In [39]:
trivia_qa_dataset = DatasetDict({
    "train": training_dataset,  # Add more splits (e.g., validation, test) as needed
    "validation": validation_dataset  # Add more splits (e.g., validation, test) as needed
})

In [41]:
print(trivia_qa_dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 138
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 18
    })
})


In [47]:
#print(trivia_qa_dataset['validation'][:3])  # Print first 3 examples

In [49]:
import torch

def preprocess_function(examples):
    print(examples["question"])
    inputs = tokenizer(
        examples["question"], examples["context"],
        max_length=512, truncation=True, padding="max_length"
    )
    
    # Extract start and end positions
    start_positions = []
    end_positions = []
    for i, answer in enumerate(examples["answers"]):
#        if len(answer["answer_start"]) > 0:
#            start = answer["answer_start"][0]
#            end = start + len(answer["text"][0])
#        else:
#            start = 0
#            end = 0
        start = 0
        end = 0
        start_positions.append(start)
        end_positions.append(end)
    
    # Convert to PyTorch tensors
    inputs["input_ids"] = torch.tensor(inputs["input_ids"])
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"])
    inputs["start_positions"] = torch.tensor(start_positions)
    inputs["end_positions"] = torch.tensor(end_positions)
    
    return inputs

In [51]:
tokenized_trivia_qa_dataset = trivia_qa_dataset.map(preprocess_function, batched=True, remove_columns=trivia_qa_dataset["train"].column_names)

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

['Where in England was Dame Judi Dench born?', 'In which decade did Billboard magazine first publish and American hit chart?', 'From which country did Angola achieve independence in 1975?', 'Which city does David Soul come from?', 'Who won Super Bowl XX?', 'Which was the first European country to abolish capital punishment?', 'In which country did he widespread use of ISDN begin in 1988?', "What is Bruce Willis' real first name?", 'Which William wrote the novel Lord Of The Flies?', 'Which innovation for the car was developed by Prince Henry of Prussia in 1911?', 'How is musician William Lee Conley better known?', 'How is Joan Molinsky better known?', 'In which branch of the arts is Patricia Neary famous?', "Which country is Europe's largest silk producer?", 'The VS-300 was a type of what?', 'At which university did Joseph Goebbels become a doctor of philosophy?', "Which prince is Queen Elizabeth II's youngest son?", "When did the founder of Jehovah's Witnesses say the world would end?"

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

['Which Lloyd Webber musical premiered in the US on 10th December 1993?', 'Who was the next British Prime Minister after Arthur Balfour?', 'Who had a 70s No 1 hit with Kiss You All Over?', 'What claimed the life of singer Kathleen Ferrier?', 'Rita Coolidge sang the title song for which Bond film?', 'What was the last US state to reintroduce alcohol after prohibition?', 'Which actress was voted Miss Greenwich Village in 1942?', 'What is the Japanese share index called?', "What was the name of Michael Jackson's autobiography written in 1988?", 'In which decade did stereo records first go on sale?', "In what year's Olympics were electric timing devices and a public-address system used for the first time?", 'Which volcano in Tanzania is the highest mountain in Africa?', 'The flag of Libya is a plain rectangle of which color?', 'Of which African country is Niamey the capital?', 'Who was the director of the CIA from 1976-81?', 'Which musical featured the song The Street Where You Live?', '"W

In [53]:
tokenized_trivia_qa_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 138
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 18
    })
})

In [55]:
tokenized_trivia_qa_dataset.set_format(type="torch")

In [57]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_trivia_qa_dataset["train"], batch_size=16, shuffle=True)
val_dataloader = DataLoader(tokenized_trivia_qa_dataset["validation"], batch_size=16)

In [59]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\Usuario\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # Number of epochs
    model.train()
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions,
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 6.387694358825684
